In [1]:
import cv2
from fer import FER
import numpy as np
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
EMOTION_FEATURE_DIR = "training_set/Features/Emotion"
FRAMES_DIR = "training_set/N-Frames"

In [3]:
def build_emotion_vector(emotions_likelihood, max_faces, emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']):
    num_faces_frame = len(emotions_likelihood)
    if num_faces_frame > max_faces: #cap the number of faces
        print(f"Video '{video_id}' has {num_faces_frame}, while max_faces is {max_faces}")
        num_faces_frame = max_faces 

    frame_feature = []
    for face in range(num_faces_frame):            # detected faces
        frame_feature.append(np.array([emotions_likelihood[face]["emotions"][emotion] for emotion in emotions]))
    for face in range(num_faces_frame, max_faces): # no faces detected, fill with zero
        frame_feature.append(np.array(np.zeros(len(emotions))))
        
    emotion_vector = np.concatenate(frame_feature)
    return emotion_vector

def print_extraction_statistics(num_faces_frames, num_faces_videos):
    num_faces_frames = np.array(num_faces_frames)
    num_faces_videos = np.array(num_faces_videos)
    print(f"{np.sum(num_faces_frames > 0)} / {len(num_faces_frames)} frames have faces")
    print(f"{np.sum(num_faces_videos > 0)} / {len(num_faces_videos)} vidoes have faces")
    print("Avg num of faces per frame:", np.mean(num_faces_frames))
    print("Max num of faces per frame:", np.max(num_faces_frames))
    print("Min num of faces per frame:", np.min(num_faces_frames))
    print("Avg num of faces per video:", np.mean(num_faces_videos))
    print("Max num of faces per video:", np.max(num_faces_videos))
    print("Min num of faces per video:", np.min(num_faces_videos))
    print("Avg num of faces per frame for frames with at least 1 face:", 
          np.mean(num_faces_frames[num_faces_frames > 1]))
    print("Avg num of faces per video for videos with at least 1 face:", 
          np.mean(num_faces_videos[num_faces_videos > 1]))
    

In [4]:
def extract_emotions(video_ids = None, frame_path = FRAMES_DIR, feature_dir = EMOTION_FEATURE_DIR, max_faces = 8):
    if video_ids is None:
        video_ids = np.sort(os.listdir(frame_path))
        
    if not os.path.exists(feature_dir):
        os.mkdir(feature_dir)
    
    failed = [] 
    detector = FER()
    
    num_faces_frames = []
    num_faces_videos = []
    
    # Extract emotion features
    for video_id in video_ids: 
        
        video_features_filename = f"{feature_dir}/{int(video_id)}.csv"
        if os.path.exists(video_features_filename):
            continue
            
        video_features = []
        num_faces_video = []
        
        for filename in os.listdir(f"{frame_path}/{video_id}"):
            emotions_likelihood = detector.detect_emotions(cv2.imread(f"{frame_path}/{video_id}/{filename}"))
            
            num_faces_frame = len(emotions_likelihood)
            num_faces_frames.append(num_faces_frame)
            num_faces_video.append(num_faces_frame)
            
            if num_faces_frame > 0: # Detected at least one face
                video_features.append(build_emotion_vector(emotions_likelihood, max_faces))
            else:
                failed.append(f"{video_id}/{filename}")
                
            
        num_faces_videos.append(np.sum(num_faces_video))
        
        if video_features:
            video_features = np.array(video_features)
            np.savetxt(video_features_filename, video_features)
            
    print_extraction_statistics(num_faces_frames, num_faces_videos)
    
    return failed

failed = extract_emotions()


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


04-11-2020:12:18:12,910 WARNING  [deprecation.py:317] From /home/tony/anaconda3/envs/video-mem/lib/python3.8/site-packages/tensorflow/python/keras/engine/training_v1.py:2070: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


(4720,)
(590,)
1563 / 4720 frames have faces
421 / 590 vidoes have faces
Avg num of faces per frame: 0.4417372881355932
Max num of faces per frame: 8
Min num of faces per frame: 0
Avg num of faces per video: 3.5338983050847457
Max num of faces per video: 55
Min num of faces per video: 0
Avg num of faces per frame for frames with at least 1 face: 2.5535714285714284
Avg num of faces per video for videos with at least 1 face: 6.057750759878419
